# Map mRNA Counts by Regions and Taxa

In [ ]:
# Important to keep this in sync whenever renaming this notebook
notebook_name = 'Ex_Histos'

# Import modules
import pymap as pm # SJS module: pymap.py must reside in current directory
import json
import shutil
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcol
import matplotlib.patches as patches
import seaborn as sns
import zipfile
from matplotlib.patches import Rectangle, Polygon
from matplotlib.backends.backend_pdf import PdfPages

# Make some choices...
dataset = 'Y23' # Y21ss','Y21tenX' or 'Y23'
geneset = '30NPPs'
# Gene Sets: 'Monoamine_Rs', '30NPPs', 'Adhesion', 'Ca_channels', 'Synaptic', '51_NP-GPCRs_Conj-Top30_NPPs'
# '61NPPs', '52NPRs', '30NPPs*Rs', '60NPPs', '30NPPs', '30NPPs*Rs', '52NPRs', '77NPRs', '137NPs'
type_id = '01 Neuronal' # 'All', '01 Neuronal', '02 Non-Neuronal' (Y21), '02 Glial' (Y23)

# Assemble the general filepaths
name_root = notebook_name
source_root = 'NP'
#source_root = 'nMod'
results_path = 'Products/'+name_root+'/'
pdf_path = results_path+'PDFs/'
csv_path = results_path+'CSVs/'
zip_path = results_path+'ZIPs/'
temp_path = 'Temp/'

# Read annotated expression data
source_path = 'Resources/'+dataset+'_'+source_root+'_counts.csv.zip' # 'NP' or 'nMod'
support_path = 'Resources/'+dataset+'_'+source_root+'_support.json' # 'NP' or 'nMod'
print('Reading',source_path)
df = pd.read_csv(source_path)

# Filter based on type ID
if type_id != 'All': anno_counts = df[df['type'] == type_id].copy()
else: anno_counts = df.copy()

# Read support dictionary
with open(support_path, 'r') as fp:
    support_dict = json.load(fp)

# Extract annotation value dictionaries
region_colors = dict(zip(anno_counts['region'],anno_counts['region_color']))
class_colors = dict(zip(anno_counts['class'],anno_counts['class_color']))
class_colors = {key: class_colors[key] for key in sorted(class_colors)}
subclass_colors = dict(zip(anno_counts['subclass'],anno_counts['subclass_color']))
subclass_colors = {key: subclass_colors[key] for key in sorted(subclass_colors)}
subclass_classes = dict(zip(anno_counts['subclass'],anno_counts['class']))
subclass_classes = {key: subclass_classes[key] for key in sorted(subclass_classes)}
subclass_shorts = dict(zip(anno_counts['subclass'],anno_counts['subclass']))
subclass_shorts = {key: subclass_shorts[key].split(' ',1)[1] for key in sorted(subclass_shorts)}
cluster_colors = dict(zip(anno_counts['cluster'],anno_counts['cluster_color']))
cluster_colors = {key: cluster_colors[key] for key in sorted(cluster_colors)}

# Read CSV table to form region dictionaries
df = pd.read_csv('Resources/'+dataset+'_region_table.csv')#[1:]
region_names = df.set_index('region_label')['region_name'].to_dict()
region_shorts = df.set_index('region_label')['region_short'].to_dict()
#region_zones = df.set_index('region_short')['zone'].to_dict()
region_zones = df.set_index('region_label')['zone'].to_dict()
zone_regions = {}
for key, value in region_zones.items():
    if value not in zone_regions:
        zone_regions[value] = [key]
    else:
        zone_regions[value].append(key)

# Form annotation unique value lists
types = (list(set(anno_counts['type']))); types.sort()
regions = list(region_colors.keys())
classes = list(class_colors.keys())
subclasses = list(subclass_colors.keys())
clusters = list(cluster_colors.keys())

# Fix up for all regions
zone_regions['All'] = regions

if dataset == 'Y23': # 'supertypes' in Y23 dataset only
    supertype_colors = dict(zip(anno_counts['supertype'],anno_counts['supertype_color']))
    supertype_colors = {key: supertype_colors[key] for key in sorted(supertype_colors)}
    supertypes = list(supertype_colors.keys())

if dataset[0:3] == "Y21": # 'neighborhood' in Y21 datasets only
    neighborhood_colors = dict(zip(anno_counts['neighborhood'],anno_counts['neighborhood_color']))
    neighborhood_colors = {key: neighborhood_colors[key] for key in sorted(neighborhood_colors)}
    neighborhoods = list(neighborhood_colors.keys())

# Read a table to form gene label dictionary
try: df = pd.read_csv('Resources/'+geneset+'.csv',usecols=[0,1], index_col=0)
except: df = pd.read_csv('Resources/All_nMods_labeled.csv', usecols=[1,2], index_col=0)
gene_labels = df.to_dict()['Label']

# Configure multiindex
major_annos = ['type','region','class','subclass','cluster']
all_annos = support_dict['annotation_labels']
minor_annos = [s for s in all_annos if s not in major_annos]
anno_counts.drop(columns=minor_annos, inplace=True)
anno_counts.set_index(major_annos, inplace=True)

# Read in a genelist and corresponding counts data
convert_to_aggrs = False if geneset[-3:] != '*Rs' else True
if convert_to_aggrs: geneset = geneset[0:-3]

genelist = list(pd.read_csv('Resources/'+geneset+'.csv')['Symbol'])   
counts = anno_counts[genelist].copy()

if convert_to_aggrs:
    # Convert an NPP matrix to an aggregated, conjugate NP-GPCR matrix
    assert 'NPP' in geneset, 'This cell requires an NPP gene list!'

    newlist_name = geneset+'*Rs'
    print('Converting', geneset, 'to', newlist_name)
    # Use conjugate pair table to generate aggregate NP_GPCRs conjugate to a set of NPPs
    # Read pair table
    pair_table = pd.read_csv('Resources/Mouse_NP_pairs.csv')
    pair_table['Pair Label'] = pair_table['NPP'] + '->' + pair_table['NP-GPCR']
    pair_id = list(pair_table['Pair ID'])
    pair_npps = list(pair_table['NPP'])
    pair_nprs = list(pair_table['NP-GPCR'])
    pair_labels = list(pair_table['Pair Label'])
    pair_table.set_index('Pair ID', inplace=True)
    # Build cognate NPP/aggregate-NP-GPCRs dictionary
    df = pair_table[['NPP','NP-GPCR']].set_index('NPP').copy()
    df['Aggs'] = df['NP-GPCR']+' '
    df = df.groupby('NPP').sum()
    aggs_dict = df['Aggs'].to_dict()
    for npp in aggs_dict: aggs_dict[npp] = aggs_dict[npp].split()
    # Build dataframe with aggregated NP-GPCRs conjugate to included NPPs
    npp_list = genelist
    df = anno_counts[npp_list].copy()
    for npp in npp_list: df[npp+'Rs'] = anno_counts[aggs_dict[npp]].sum(axis=1)
    npr_list = [x+'Rs' for x in npp_list]
    counts = df.copy()

    geneset = newlist_name
    genelist = npr_list
    counts = counts[npr_list].copy()

print('Genelist:',geneset,', Counts matrix shape:', counts.shape, '\nResults path: ', results_path)
gene_order = genelist


In [ ]:
zone_regions

In [ ]:
# Make choices
gene = 'Cck'
zone_id = 'Neocortex' # 'All', 'Neocortex', 'Allocortex', 'Basal', 'Mesen', 'Rhomben'
class_id = 'All'
#class_id = '01 Glutamatergic'
#class_id = '02 GABAergic'
unit = 'Clusters' # 'Cells' or 'Clusters'
min_cluster = 10

# Slice counts data
df = counts.copy()
df = df[gene]
df = df.droplevel(['type'])
df = df.loc[zone_regions[zone_id]]
df = df.droplevel(['region'])
if class_id != 'All': df = df.loc[class_id]
df = df.droplevel(['subclass'])

# Form clusters if called for
cells_pre = len(df)
clusters_pre = len(df.groupby('cluster').count())
df = df.to_frame().groupby(['cluster']).filter(lambda x: len(x) >= min_cluster) # filter out small clusters
cells_post = len(df)
clusters_post = len(df.groupby('cluster').count())
print(cells_pre - cells_post, 'cells filtered from', cells_pre, 'due due to cluster size <', min_cluster)
print(clusters_pre - clusters_post, 'clusters filtered from', clusters_pre, 'due due to cluster size <', min_cluster)

if unit == 'Clusters': df = df.groupby(['cluster']).mean()
    
histo,bins,artists = plt.hist(df, rwidth=0.75, align='left', bins=50, range=(0,500))
plt.ylim(0, 1.05*histo[1])
plt.title(dataset+'  '+type_id+'  '+gene+'\n'+str(round(100*(len(df)-histo[0])/len(df),1))+
          '% '+unit+' above zero, Highest Count = '+str(max(df)), fontsize=10)
plt.xlabel(gene+' Counts')
plt.ylabel('Number of '+unit)
plt.show()

histo,bins,artists = plt.hist(df, rwidth=0.75, align='left', bins=50, range=(0,2))
plt.ylim(0, 1.05*max(histo))
plt.title(dataset+'  '+type_id+'  '+gene+'\n'+str(round(100*(len(df)-histo[0])/len(df),1))+
          '% '+unit+' above zero, Highest Count = '+str(max(df)), fontsize=10)
plt.xlabel(gene+' Counts')
plt.ylabel('Number of '+unit)
plt.show()


In [ ]:
# Co-expression by clusters

# Make some choices
threshold = 1
min_cluster = 20
zone = 'Neocortex' # 'All', 'Neocortex', 'Allocortex', 'Basal', 'Mesen', 'Rhomben'

# Generate filename root
root = dataset
if type_id == 'All': root += '_all_'
elif type_id == '01 Neuronal': root += '_neur_'
else: root += type_id+'_'
root += geneset+'_'

# Get a serial number and save a copy of this notebook
sn = pm.serial(5)
source_notebook_path = notebook_name+'.ipynb'
destination_notebook_path = 'Notebooks/'+notebook_name+'_'+sn+'.ipynb'
shutil.copy2(source_notebook_path, destination_notebook_path)

# Form titling/naming handle
handle = root+zone+'_T'+str(threshold)+'_M'+str(min_cluster)+'_'+sn
pdf_file = pdf_path+handle+'.pdf'
pdf = PdfPages(pdf_file)

regions = zone_regions[zone].copy()
regions.insert(0, 'All')

for region in regions:
    df = counts.copy()
    df = df.droplevel(['type','class','subclass'])
    if region == 'All': 
        if zone != 'All': df = df.loc[zone_regions[zone]]
    else:
        df = df.loc[region]
    cells_pre = len(df)
    clusters_pre = len(df.groupby('cluster').count())
    df = df.groupby(['cluster']).filter(lambda x: len(x) >= min_cluster) # filter out small clusters
    cells_post = len(df)
    clusters_post = len(df.groupby('cluster').count())
    print(cells_pre - cells_post, 'cells filtered from', cells_pre, 'due due to cluster size <', min_cluster)
    print(clusters_pre - clusters_post, 'clusters filtered from', clusters_pre, 'due due to cluster size <', min_cluster)
    cluster_mean = df.groupby(['cluster']).mean()
    cluster_hot = (cluster_mean >= threshold).astype(int)

    npps = cluster_hot.T
    any_npp = npps.sum()
    histo,bins,artists = plt.hist(any_npp, bins=12, range=(0,12), rwidth=0.9)

    plt.xlabel('# NPPs Co-Expressed\n'+str(histo))
    plt.ylabel('Frequency')
    plt.title('zone: '+zone+', region: '+region+', threshold: '+str(threshold)+', '+
              str(round(100*histo[0]/len(any_npp),2))+'% zeroes\n'+
              'min cluster: '+str(min_cluster)+', '+str(cells_pre-cells_post)+' / '+str(cells_pre)+' cells filtered\n'+
              str(clusters_pre-clusters_post)+' / '+str(clusters_pre)+' clusters filtered, '+
              str(clusters_post)+' clusters remain', fontsize=10)

    # Save and show the plot
    pdf.savefig(dpi=300, transparent=True, bbox_inches='tight', pad_inches=0.2)
    plt.show()
    
pdf.close(); print('Graphics to PDF:', pdf_file)

In [ ]:
# Co-expression by cells

# Make some choices
threshold = 1
zone = 'Neocortex' # 'All', 'Neocortex', 'Allocortex', 'Basal', 'Mesen', 'Rhomben'

# Generate filename root
root = dataset
if type_id == 'All': root += '_all_'
elif type_id == '01 Neuronal': root += '_neur_'
else: root += type_id+'_'
root += geneset+'_'

# Get a serial number and save a copy of this notebook
sn = pm.serial(5)
source_notebook_path = notebook_name+'.ipynb'
destination_notebook_path = 'Notebooks/'+notebook_name+'_'+sn+'.ipynb'
shutil.copy2(source_notebook_path, destination_notebook_path)

# Form titling/naming handle
handle = root+zone+'_T'+str(threshold)+'_'+sn

pdf_file = pdf_path+handle+'.pdf'
pdf = PdfPages(pdf_file)

regions = zone_regions[zone].copy()
regions.insert(0, 'All')

for region in regions:
    df = counts.copy()
    df = df.droplevel(['type','class','subclass'])
    if region == 'All': 
        if zone != 'All': df = df.loc[zone_regions[zone]]
    else:
        df = df.loc[region]
        
    cell_hot = (df >= threshold).astype(int)
    npps = cell_hot.T
    any_npp = npps.sum()
    histo,bins,artists = plt.hist(any_npp, bins=12, range=(0,12), rwidth=0.9)

    plt.xlabel('# NPPs Co-Expressed\n'+str(histo))
    plt.ylabel('Frequency')
    plt.title('zone: '+zone+', region: '+region+', threshold: '+str(threshold)+', '+
              str(round(100*histo[0]/len(any_npp),2))+'% zeroes',fontsize=10)

    # Save and show the plot
    pdf.savefig(dpi=300, transparent=True, bbox_inches='tight', pad_inches=0.2)
    plt.show()

pdf.close(); print('Graphics to PDF:', pdf_file)